In [7]:
import numpy
import os
import cv2
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import Adam

In [28]:
DATA = {}
PATH = "./data/PlantVillage/"
DEFAULT_IMG_SIZE = tuple((256,256))
X = []
Y = []
KEY  = {}

In [ ]:
def toDict():
    folders = os.listdir(PATH)
    img_list = []
    c = 0
    for folder in folders:
        print("[INFO] PROCESSING " + folder)
        if os.path.isdir(PATH+folder):
            total = len(os.listdir('{}{}'.format(PATH,folder)))
            count = 0
            images = os.listdir("{}{}".format(PATH,folder))
            KEY[folder] = c
            c += 1
            img_list = []
            for image in images:
                count += 1
                print("\r[INFO] COMPLETED = {}%".format(count*100//total),end = '')
                img = cv2.imread("{}{}/{}".format(PATH,folder,image))
                if img is not None :
                    if img.shape != DEFAULT_IMG_SIZE:
                        img = cv2.resize(img,DEFAULT_IMG_SIZE)
                    img_list.append(img.astype('float32')/255)
            DATA[folder] = img_list
        
toDict()   

[INFO] PROCESSING Tomato__Tomato_YellowLeaf__Curl_Virus
[INFO] COMPLETED = 36%

In [36]:
for key, value in DATA.items():
    for v in value:
        X.append(v)
        Y.append(KEY[key])
X = numpy.array(X)
Y = numpy.array(Y)

In [46]:
DATA['Potato___healthy'][]

array([[[0.4       , 0.38431373, 0.43137255],
        [0.3647059 , 0.34901962, 0.39607844],
        [0.4       , 0.38431373, 0.43137255],
        ...,
        [0.6       , 0.58431375, 0.627451  ],
        [0.6       , 0.58431375, 0.627451  ],
        [0.60784316, 0.5921569 , 0.63529414]],

       [[0.38431373, 0.36862746, 0.41568628],
        [0.40392157, 0.3882353 , 0.43529412],
        [0.4117647 , 0.39607844, 0.44313726],
        ...,
        [0.5882353 , 0.57254905, 0.6156863 ],
        [0.5921569 , 0.5764706 , 0.61960787],
        [0.6       , 0.58431375, 0.627451  ]],

       [[0.3529412 , 0.3372549 , 0.38431373],
        [0.43137255, 0.41568628, 0.4627451 ],
        [0.45882353, 0.44313726, 0.49019608],
        ...,
        [0.5921569 , 0.5764706 , 0.61960787],
        [0.6       , 0.58431375, 0.627451  ],
        [0.6117647 , 0.59607846, 0.6392157 ]],

       ...,

       [[0.48235294, 0.46666667, 0.49019608],
        [0.49411765, 0.47843137, 0.5019608 ],
        [0.5372549 , 0